# Planet Destinations Python Client Introduction

This tutorial is an introduction to [Planet's](https://www.planet.com) Destination API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `Planet` module. The Destination API allows you manage and securely store cloud storage bucket credentials as a 'Destination' for a product delivery on the Planet Platform. This can service as an alternative to adding credentials to multiple workflows, for example, as the Destination can be setup once, and then referenced in different Order or Subscription requests.

## Requirements

An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's API's. If you are not logged in, you will be prompted to do so below in the *SDK Authentication* section.

## Useful links 
* [Planet SDK for Python](https://planet-sdk-for-python.readthedocs.io/en/stable/get-started/quick-start-guide/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Destinations Documentation](https://docs.planet.com/develop/apis/destinations/)

The basic workflow for interaction with the Destinations API is:
1. Prepare destination variables; mainly credentials and bucket name, with some other items depending on cloud storage provider.
2. Create Destination using the Destinations API
3. Use the Destination Reference ID in Orders and Subscription Requests.

____

## <u>Destination Information</u>

#### Supported Cloud Storage Destinations:
* Amazon S3
    * Any other service that implements the S3 compatibility API
* Google Cloud Storage
* Microsoft Azure Blob Storage
* Oracle Cloud Storage


#### Detinations Limitations:
* Each *organization* is provided a limitation of up to 50 Destinations, which will be both visible and usable by each individual within said organization.
* The creator of a Destination is considered its *owner*, and, along with your organizational administrator, will be the only one allowed to modify a Destination.
* All Destinations must have unique names.
* All Destinations are rate limited to 3 requests per second.

#### Destination Credentials
* Credentials used to create a Destination must have both `write` and `delete` permissions (the specific names of these permissions may vary).
    * When a destination is created or updated, Planet verifies access by attempting to write and then delete a test file named `planetverify.txt`. 
    * If the permissions are correctly configured, this file will be removed immediately and will not appear in the storage bucket or container.* If you ever see this file, your credentials may be lacking the `delete` permissions needed.

#### Credential Security
* Destination credentials are treated as secrets which are encrypted at rest and in transit between Planet systems, accessed and decrypted only when strictly required, during a delivery.
* Destinations API will always redact these secrets in its responses.
    * You can update credentials via the Destination API, but never read them using it.


____

## Setup

In order to interact with the Planet Destinations API using the Python client, we need to import the necessary packages and authenticate our Planet account credentials.

### Imports

In [8]:
import json
import pathlib
import planet
import getpass
import os
from datetime import datetime
from planet import Auth, Planet, Session, DestinationsClient
# Add additional imports as needed

### SDK Authentication

Your Planet login is used to authenticate and activate the Python SDK. You will be prompted via a link below to login and confirm on the page that the code displayed matches the authorization code printed. If this is your first time accessing the Planet SDK, you will also be prompted first to authorize the SDK access to your Planet account. 
If you would like to know more, please visit the [authentication documentation](https://docs.planet.com/develop/authentication).

In [ ]:
# Authenticate for the Planet SDK; See docs: https://docs.planet.com/develop/authentication
# If you are not already logged in, this will prompt you to open a web browser to log in.

auth = planet.Auth.from_profile('planet-user', save_state_to_storage=True)
auth.ensure_initialized(allow_open_browser=False, allow_tty_prompt=True)

session = planet.Session(auth)
pl = planet.Planet(session)

_____

### Prepare Destination Credentials

Before you create a Destination, you must first complete the formation of a credentials dictionary. While this has slight variations between cloud provider, this will always have a `name`, a `type` (provider), with `parameters` that will contain the `bucket name` and some sort of `credential keys`. 

For convenience, placeholders for the various supported cloud storage providers will be in the cells below. The names of the specific permissions with also be listed.

The various secret keys for credentials are meant to be placeholders, and can be set as environment variables to avoid pasting them in anywhere. Alternatively, they can be entered using `getpass` below.

In [ ]:
# Use this variable in place of things like aws_secret_access_key, credentials_str, sas_token, etc.
credential_secret = getpass.getpass('credential secret:')

#### <u>Amazon S3</u>
For Amazon S3 delivery use an AWS account with `GetObject`, `PutObject`, and `DeleteObject` permissions.

You will also need a `aws_region`, `aws_access_key_id`, and `aws_secret_access_key`.

In [ ]:
name = 's3 destination'
bucket = 'bucket name'
aws_region = 'region name'
aws_access_key_id = 'access key id'
aws_secret_access_key = 'aws secret access key'

bucket_creds = {
    "name": name,
    "type": "amazon_s3",
    "parameters": {
        "bucket": bucket,
        "aws_region": aws_region,
        "aws_access_key_id": aws_access_key_id,
        "aws_secret_access_key": aws_secret_access_key,
    },
}

#### <u> Google Cloud Storage </u>

For Google Cloud Storage delivery, a service account with `storage.objects.create`, `storage.objects.get`, and `storage.objects.delete` permissions is required.

***Important***: The Google Cloud Storage delivery option requires a single-line base64 version of the service account credentials for use by the credentials parameter.

Download the service account credentials in JSON format (not P12) and encode them as a `base64` string, use a command line operation such as:

```bash
cat my_creds.json | base64
```

In [ ]:
name = 'GCS destination'
bucket_name = 'bucket name'
credentials_str = 'base64 str'

bucket_creds = {
    "name": name,
    "type": "google_cloud_storage",
    "parameters": {
        "bucket": bucket_name,
        "credentials": credentials_str,
    },
}

#### <u> Microsoft Azure Blob Storage </u>

For Microsoft Azure delivery use an Azure account with `read`, `write`, `delete`, and `list` permissions.

You will also need: `account name`, `container_name`, and `sas_token`.

In [ ]:
name = 'azure destination'
account = 'account name'
container_name = 'container name'
sas_token = 'token'

# Optional: storage endpoint suffix:
# storage_endpoint_suffix = 'storage endpoint suffix'

bucket_creds = {
    "name": name,
    "type": "google_cloud_storage",
    "parameters": {
        "account": account,
        "container": container_name,
        "sas_token": sas_token,
        #   "storage_endpoint_suffix": storage_endpoint_suffix, # optional
    },
}

#### <u> Oracle Cloud Storage </u>
For Oracle Cloud Storage delivery, use an Oracle account with `read`, `write`, and `delete` permissions.

For authentication, use a `Customer Secret Key` which consists of an `Access Key/Secret Key` pair.

In [ ]:
name = 'ocs name'
bucket_name = 'bucket name'
region = 'region'
namespace = 'namespace'
customer_access_key_id = 'customer_access_key_id'
customer_secret_key = 'customer_secret_key'

bucket_creds = {
    "name": name,
    "type": "oracle_cloud_storage",
    "parameters": {
        "bucket": bucket_name,
        "region": region,
        "namespace": namespace,
        "customer_access_key_id": customer_access_key_id,
        "customer_secret_key": customer_secret_key,
    },
}

#### <u> Other S3 Compatible Storage Providers </u>


To use this delivery method, use an account with `read`, `write`, and `delete` permissions for the bucket.

Authentication is performed using an Access Key and Secret Key pair.

You will also need `endpoint`, and can optionally use a path style.
* Pay attention to the `use_path_style` parameter if you choose a path style, as it is a common source of issues.
    * For example, Oracle Cloud requires use_path_style to be true, while Open Telekom Cloud requires it to be false.

In [ ]:
name = 's3 compatible destination'
bucket = 'bucket name'
region = 'region name'
endpoint = 'endpoint'
access_key_id = 'access key id'
secret_access_key = 'secret access key'

# Optional:
# use_path_style = False

bucket_creds = {
    "name": name,
    "type": "s3_compatible",
    "parameters": {
        "bucket": bucket,
        "region": region,
        "endpoint": endpoint,
        "access_key_id": access_key_id,
        "secret_access_key": secret_access_key,
        # "use_path_style": use_path_style
    },
}

___

___

### Destination Creation

Once you have setup your credentials dictionary above, you should use it to create a Destination:

In [ ]:
pl.destinations.create_destination(bucket_creds)

Signature: pl.destinations.create_destination(request: Dict[str, Any]) -> Dict
Docstring:
Create a new destination.

Args:
    request (dict): Destination content to create, all attributes are required.

Returns:
    dict: A dictionary containing the created destination details.

Raises:
    APIError: If the API returns an error response.
    ClientError: If there is an issue with the client request.
File:      ~/dev_env/public_repos/notebooks/.venv/lib/python3.13/site-packages/planet/sync/destinations.py
Type:      method

In [ ]:
Destination Reference ID's

____

### Section 3

[Add section description here]

In [ ]:
# Add your code here

_____

### Section 4

[Add section description here]

In [ ]:
# Add your code here

___

### Section 5

[Add section description here]

In [ ]:
# Add your code here